In [1]:
%matplotlib inline

In [2]:
#Imports
import numpy as np
import pandas as pd

# Heart Disease Analysis and Prediction - Draft

### 1. Data Acquisition

In [3]:
heart_data = pd.read_csv("data/heart.csv")
heart_data.info() # Remove this later

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       303 non-null    int64  
 1   sex       303 non-null    int64  
 2   cp        303 non-null    int64  
 3   trestbps  303 non-null    int64  
 4   chol      303 non-null    int64  
 5   fbs       303 non-null    int64  
 6   restecg   303 non-null    int64  
 7   thalach   303 non-null    int64  
 8   exang     303 non-null    int64  
 9   oldpeak   303 non-null    float64
 10  slope     303 non-null    int64  
 11  ca        303 non-null    int64  
 12  thal      303 non-null    int64  
 13  target    303 non-null    int64  
dtypes: float64(1), int64(13)
memory usage: 33.3 KB


In [4]:
# Shape - describe how many observations and variable do we have
heart_data.shape

(303, 14)

In [5]:
# Display several observation in order to get a sence of the data -> Tell what you see in one, two sentances
heart_data.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [6]:
heart_data.tail(3)

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
300,68,1,0,144,193,1,1,141,0,3.4,1,2,3,0
301,57,1,0,130,131,0,1,115,1,1.2,1,1,3,0
302,57,0,1,130,236,0,0,174,0,0.0,1,1,2,0


In [7]:
heart_data.sample(5)

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
298,57,0,0,140,241,0,1,123,1,0.2,1,0,3,0
294,44,1,0,120,169,0,1,144,1,2.8,0,0,1,0
221,55,1,0,140,217,0,1,111,1,5.6,0,0,3,0
100,42,1,3,148,244,0,0,178,0,0.8,2,2,2,1
67,45,0,1,130,234,0,0,175,0,0.6,1,0,2,1


In [8]:
# Columns -> Rename columns and after that describe the variables what they mean -> see the documentation and the research info too
#TODO - rename some columns
heart_data.columns = [
    "age", "sex", "chest_pain_type", "rest_blood_pressure", "cholesterol", "blood_sugar", "ecg_rest", "heart_rate_max",
    "exercise_angina", "oldpeak", "ST_slope", "vessels", "thalassemia", "heart_disease"]

In [9]:
# Data types -> Describe the data types of the variables but tell also that there are many categorical variables
# Check ranges and value validity of each variable step by step clean -> Cleaning
heart_data.dtypes

age                      int64
sex                      int64
chest_pain_type          int64
rest_blood_pressure      int64
cholesterol              int64
blood_sugar              int64
ecg_rest                 int64
heart_rate_max           int64
exercise_angina          int64
oldpeak                float64
ST_slope                 int64
vessels                  int64
thalassemia              int64
heart_disease            int64
dtype: object

In [10]:
# Age
heart_data["age"].min(), heart_data["age"].max() # -> It looks valid

(29, 77)

In [11]:
# Sex
heart_data["sex"].unique()  # Interpret the values

array([1, 0], dtype=int64)

In [12]:
# Transform sex to categorical variable
def modify_sex(sex_numeric):
    return "Male" if sex_numeric == 1 else "Female"

transformed_sex = heart_data["sex"].apply(modify_sex)
heart_data["sex"] = pd.Categorical(transformed_sex)

In [13]:
# Chest pain type cleaning

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype   
---  ------               --------------  -----   
 0   age                  303 non-null    int64   
 1   sex                  303 non-null    category
 2   chest_pain_type      303 non-null    int64   
 3   rest_blood_pressure  303 non-null    int64   
 4   cholesterol          303 non-null    int64   
 5   blood_sugar          303 non-null    int64   
 6   ecg_rest             303 non-null    int64   
 7   heart_rate_max       303 non-null    int64   
 8   exercise_angina      303 non-null    int64   
 9   oldpeak              303 non-null    float64 
 10  ST_slope             303 non-null    int64   
 11  vessels              303 non-null    int64   
 12  thalassemia          303 non-null    int64   
 13  heart_disease        303 non-null    int64   
dtypes: category(1), float64(1), int64(12)
memory usage: 31.3 KB
